In [4]:
# !pip install faiss-cpu

In [29]:
# Download stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.tokenize import sent_tokenize
import datasets
import json

# R_NUMBER_SEED = 1234567 # Replace this with your own student number
R_NUMBER_SEED = 928036 # my student number 
DOCS_TO_ADD = 1000
query_documents = datasets.load_dataset("parquet", data_files="./acl_anthology_queries.parquet")["train"]
all_documents = datasets.load_dataset("parquet", data_files="./acl_anthology_full.parquet")["train"]
# Shuffle with seed and take only n docs
shuffled_documents = all_documents.shuffle(seed=R_NUMBER_SEED)
random_documents = shuffled_documents.select(range(DOCS_TO_ADD))
# Concatenate relevant documents with random sample and shuffle again
anthology_sample = datasets.concatenate_datasets([query_documents, random_documents]).shuffle(seed=R_NUMBER_SEED)
# Export to Parquet to avoid downloading full anthology
anthology_sample.to_parquet("./anthology_sample.parquet")

queries = json.load(open("./acl_anthology_queries.json", "r"))


# Remove stopwords
def remove_stopwords(doc):
    text = ' '.join([word for word in doc.split() if word.lower() not in stop_words])

def preprocess_document(doc):
    # Flatten dictionary and combine relevant text fields
    # text = f"{doc.get('title', '')} {doc.get('abstract', '')} {doc.get('full_text', '')}"
    text = f"{doc.get('full_text', '')}"
    
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    
    return text
def preprocess_document_using_ntlk_sent(doc):
    full_text = f"{doc.get('full_text', '')}"
    sentences = sent_tokenize(full_text)
    return sentences

def preprocess_documents(documents):
    all_sentences = []
    all_sentences_to_doc_map = []
    for doc in documents:
        print(doc["acl_id"])
        full_text = f"{doc.get('full_text')}"
        sentences = sent_tokenize(full_text)
        for sentence in sentences:
            all_sentences.append(sentence)
            all_sentences_to_doc_map.append(doc["acl_id"])
    return all_sentences,all_sentences_to_doc_map

preprocessed_documents,preprocess_documents_map = preprocess_documents(anthology_sample)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mjova\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

L16-1030
W09-3723
N19-1170
C92-1024
P90-1005
S07-1030
C18-1094
W02-0905
W98-0306
W11-2027
W03-3010
2020.coling-main.175
W16-2118
W11-1416
2020.emnlp-demos.1
N16-3020
J78-2007
2020.acl-main.215
W10-2210
2022.findings-acl.13
W16-5316
N16-1024
W18-6459
E99-1023
L14-1197
W97-0715
2018.iwslt-1.26
W08-2122
2021.acl-long.346
P18-1040
D17-1161
W03-2115
2008.jeptalnrecital-recital.6
2022.lchange-1.7
P87-1001
W10-1734
2022.naacl-main.67
C82-1060
C98-2195
1996.tc-1.3
N07-1034
W14-0106
D19-1375
1952.earlymt-1.10
W15-2917
2021.tacl-1.92
W09-3025
2022.findings-acl.137
2020.nlpcss-1.19
2011.mtsummit-papers.51
D17-1042
W10-3403
N18-1108
W13-4202
C16-1024
2022.findings-emnlp.61
2020.acl-main.240
W14-2212
2022.naacl-main.72
W19-8617
W09-0623
P17-3006
L08-1137
P17-1019
L16-1674
S19-2100
J00-3008
E12-2016
L02-1136
W17-1721
2020.findings-emnlp.292
1993.iwpt-1.7
O16-1024
H90-1012
Y06-1051
D18-1282
1998.amta-papers.22
P19-1334
W19-4107
2020.lrec-1.463
D17-1005
D18-1273
W12-2109
W08-0909
2020.acl-main.335
202

In [34]:
len(preprocessed_documents)

185801

In [35]:
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
import faiss
import numpy as np
import time

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example documents with structure
documents = [
    {"id": "doc1", "title": "Title 1", "abstract": "Abstract 1", "full_text": "Sentence 1 of doc 1. Sentence 2 of doc 1."},
    {"id": "doc2", "title": "Title 2", "abstract": "Abstract 2", "full_text": "Sentence 1 of doc 2. Sentence 2 of doc 2."},
    {"id": "doc3", "title": "Title 3", "abstract": "Abstract 3", "full_text": "Sentence 1 of doc 3. Sentence 2 of doc 3."},
    {"id": "doc4", "title": "Title 4", "abstract": "Abstract 4", "full_text": "Sentence 1 of doc 4. Sentence 2 of doc 4."}
]

# Split full_text into sentences and create a mapping to document ids
all_sentences = []
sentence_to_doc_map = []

# for doc in documents:    
#     sentences = doc["full_text"].split(". ")  # Assuming sentences are separated by ". "
#     for sentence in sentences:
#         all_sentences.append(sentence)
#         sentence_to_doc_map.append(doc["id"])

# Create embeddings
embeddings = model.encode(preprocessed_documents)
print("Embeddings created.")


Embeddings created.
KNN Results:


IndexError: list index out of range

In [41]:
# Initialize KNN
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
# Fit the model
knn.fit(embeddings)

NearestNeighbors(metric='cosine')

In [42]:
# Example query
query = queries["queries"][50]['q']

query_embedding = model.encode([query])
print("Ground truth question:", queries["queries"][50]['q'])
print("Ground truth answer:", queries["queries"][50]['a'])
print("Ground truth references:", queries["queries"][50]['r'])

Ground truth question: What is the name of the research initiative creating resources for African languages?
Ground truth answer: Masakhane. (Masakha is also fine.)
Ground truth references: ['2023.acl-long.796', '2023.acl-long.609', '2023.ijcnlp-main.10']


In [49]:

# Perform KNN search
nn = 5
start_time = time.time()
distances, knn_indices = knn.kneighbors(query_embedding,n_neighbors=nn)
knn_time = time.time() - start_time


In [50]:
print("KNN Results:")
for index in knn_indices[0]:
    print(f"Document ID: {preprocess_documents_map[index]}, Sentence: {preprocessed_documents[index]}, Distance: {distances[0][knn_indices[0].tolist().index(index)]}")
print(f"KNN search time: {knn_time} seconds")


KNN Results:
Document ID: 2023.acl-long.609, Sentence: First, it is the second most available resource after the religious domain for most African languages., Distance: 0.3035791516304016
Document ID: 2021.gem-1.10, Sentence: Chris Emezue and Rubungo Andre Niyongabo explored potential low-resource African languages for crowdsourcing., Distance: 0.33035075664520264
Document ID: 2023.ijcnlp-main.10, Sentence: Data Annotation We recruited volunteers from the Masakhane community-an African grassroots community focused on advancing NLP for African languages., Distance: 0.33780044317245483
Document ID: 2023.acl-long.609, Sentence: While great strides have been made for (major) Indo-European languages such as English, French and German, work on the African languages is quite scarce., Distance: 0.33969610929489136
Document ID: 2022.coling-1.382, Sentence: An example for this approach is AfriB-ERTa (Ogueji et al., 2021) , which was trained from scratch on 11 African languages., Distance: 0.3420

In [51]:
# Initialize FAISS index
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to FAISS index
faiss_index.add(embeddings)
print("FAISS index built.")

# Perform FAISS search
start_time = time.time()
D, I = faiss_index.search(query_embedding, nn)
faiss_time = time.time() - start_time

print("FAISS Results:")
for i, index in enumerate(I[0]):
    print(f"Document ID: {preprocess_documents_map[index]}, Sentence: {preprocessed_documents[index]}, Distance: {D[0][i]}")
print(f"FAISS search time: {faiss_time} seconds")


FAISS index built.
FAISS Results:
Document ID: 2023.acl-long.609, Sentence: First, it is the second most available resource after the religious domain for most African languages., Distance: 0.6071585416793823
Document ID: 2021.gem-1.10, Sentence: Chris Emezue and Rubungo Andre Niyongabo explored potential low-resource African languages for crowdsourcing., Distance: 0.660702109336853
Document ID: 2023.ijcnlp-main.10, Sentence: Data Annotation We recruited volunteers from the Masakhane community-an African grassroots community focused on advancing NLP for African languages., Distance: 0.675601065158844
Document ID: 2023.acl-long.609, Sentence: While great strides have been made for (major) Indo-European languages such as English, French and German, work on the African languages is quite scarce., Distance: 0.6793924570083618
Document ID: 2022.coling-1.382, Sentence: An example for this approach is AfriB-ERTa (Ogueji et al., 2021) , which was trained from scratch on 11 African languages., 